In [ ]:
from contextlib import suppress

import numpy as np
import pandas as pd
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from pathlib import Path

from requests import get, Response
from hashlib import sha256
from tqdm.notebook import tqdm
from zipfile import ZipFile
from IPython.display import display, Markdown
import pandas as pd
from utils.image_inverter import save as save
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# For pretty writing
from IPython.display import display, Markdown


# Analyse exploratoire de données (Exploratory Data Analysis)


## Partie 0 - Outils

In [ ]:


graph_folder: Path = Path("./graphs")


def save_figure(figure: plt.Figure, folder: str, figure_name: str) -> None:
    folder = graph_folder / folder
    folder.mkdir(parents=True, exist_ok=True)
    save(figure, folder / f'{figure_name}.png')


_cache_folder = Path('~/.cache/gn_p7').expanduser()
_cache_folder.mkdir(parents=True, exist_ok=True)

_ds_url = 'https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/Parcours_data_scientist/Projet+-+Impl%C3%A9menter+un+mod%C3%A8le+de+scoring/Projet+Mise+en+prod+-+home-credit-default-risk.zip'


def download(url: str) -> Path:
    url_id: str = sha256(url.encode('utf-8')).hexdigest()
    local_path: Path = _cache_folder / url_id
    local_path.parent.mkdir(parents=True, exist_ok=True)
    if not local_path.exists():
        tmp_path: Path = _cache_folder / (url_id + '.tmp')
        res: Response = get(url, stream=True)
        with tmp_path.open('wb') as f, tqdm(
                total=int(res.headers.get('content-length')),
                desc=f'Downloading {url}',
                unit_scale=True) as q:
            for chunk in res.iter_content(chunk_size=8192):
                q.update(len(chunk))
                f.write(chunk)
        tmp_path.replace(local_path)
    return local_path


def download_zip_archive(url: str) -> Path:
    """Download a zip archive, extract it then return the folder containing its content"""
    archive_path: Path = download(url)
    archive_folder: Path = Path(archive_path.as_posix() + '.dir')

    if not archive_folder.exists():
        print(f'Extracting archive {url}...', flush=True)
        archive_temp: Path = Path(archive_path.as_posix() + '.tmp')
        archive_temp.mkdir(parents=True, exist_ok=True)
        archive: ZipFile = ZipFile(archive_path)
        archive.extractall(path=archive_temp)
        archive_temp.replace(archive_folder)
        print(f'Extracting archive {url}...done', flush=True)

    return archive_folder


datasets: dict[str, pd.DataFrame] = {}


def get_dataset(name: str) -> pd.DataFrame:
    folder = download_zip_archive(_ds_url)
    if not name.endswith('.csv'):
        name = f'{name}.csv'
    try:
        return datasets[name]
    except KeyError:
        try:
            _df = pd.read_csv(folder / name)
        except FileNotFoundError:
            display(Markdown(f'# ERROR: Dataset {name!r} not found, available datasets are:\n' + '\n'.join(
                f'- {p.name}' for p in sorted(folder.iterdir(), key=(lambda x: x.name.lower())))))
            raise KeyError(name) from None
        else:
            datasets[name] = _df
            return _df.copy()


## Partie 1 - Applications
Notre dataset contiens plusieurs fichiers.
Nous feront comme le notebook exemple, en commençant initialement avec application_{train,test}, puis en rajoutant des données pour améliorer la performance du modèle.

### Partie 1.1 - Données brutes

Il serai peut-être nécéssaire de prétraiter les données, cependant je vais tenter de créer un modèle initial sur les données brutes pour servir de référence

### 1.1.1 - k-Means

k-Means est linéaire, et non supervisé, il aura probablement un score assez faible

In [ ]:
# Scoring related



def heat_map(matrix: np.ndarray, classes: list[str], title: str, y_label: str, x_label: str, fmt: str) -> None:
    sns.heatmap(matrix, annot=True, fmt=fmt, xticklabels=classes, yticklabels=classes, cmap='Blues')
    plt.title(title, fontsize=16)
    plt.ylabel(y_label, fontsize=12)
    plt.xlabel(x_label, fontsize=12)
    plt.show()

def confusion(
        y_truth: pd.Series,
        y_prediction: pd.Series,
        model_name: str,
) -> dict:
    assert len(y_truth) == len(y_prediction), "Input Series must have the same length."
    classes = sorted(set(y_truth) | set(y_prediction))
    matrix = confusion_matrix(y_truth, y_prediction, labels=classes)
    sns.heatmap(matrix / len(y_truth), annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title(f'Confusion Matrix for {model_name!r}', fontsize=16)
    plt.ylabel('Actual Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    return {
        # TODO: More metrics
        'accuracy': accuracy_score(y_truth, y_prediction),
        'precision': precision_score(y_truth, y_prediction, zero_division=0),
        'recall': recall_score(y_truth, y_prediction, zero_division=0),
        'f1_score': f1_score(y_truth, y_prediction, zero_division=0)
    }
def confusion(
        y_truth: pd.Series,
        y_prediction: pd.Series,
        model_name: str,
) -> dict:
    assert len(y_truth) == len(y_prediction), "Input Series must have the same length."
    classes = sorted(set(y_truth) | set(y_prediction))
    heat_map(
        (matrix := confusion_matrix(y_truth, y_prediction, labels=classes)),
        classes, f'Confusion Matrix for {model_name!r} (absolute values)', 'Actual Label', 'Predicted Label', 'd')

    # 'matrix' est la matrice de confusion non normalisée
    # On divise chaque valeur par la somme de sa ligne
    # np.newaxis est crucial pour que la division se fasse correctement
    heat_map(
        (100 * (matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis])),
        classes, f'Confusion Matrix for {model_name!r} (percentages)', 'Actual Label', 'Predicted Label', '.02f')

    return {
        # TODO: More metrics
        'accuracy': accuracy_score(y_truth, y_prediction),
        'precision': precision_score(y_truth, y_prediction, zero_division=0),
        'recall': recall_score(y_truth, y_prediction, zero_division=0),
        'f1_score': f1_score(y_truth, y_prediction, zero_division=0)
    }


def k_means():
    display(Markdown('# WARNING: k-Means distinguish between two classes, but might switch classes'))
    X = get_dataset('application_train.csv')
    y_true = X.pop('TARGET')
    for column in list(X.columns):
        try:
            X[column] = X[column].astype(float).fillna(0)
        except ValueError:
            X.pop(column)
    model = KMeans(len(y_true.unique()))
    y_pred = model.fit(X).predict(X)
    return confusion(y_true, y_pred, 'k-Means')


k_means()

In [ ]:
# Similar to plt.hist, but automatically labels the axes
plt.show(sns.countplot(data=(tmp := get_dataset('application_train.csv')), x='TARGET').set_title(
    'Distribution of Target Values in the training dataset').figure)

display(Markdown(
    '<b>Nous pouvons voir ici un clair déséquilibre des classes, en effet, '
    f'seuls {100 * sum(tmp.TARGET) / len(tmp.TARGET):.2f}% des clients ont difficultés de paiement</b><br>'
))

### 1.1.2 Start Here: A Gentle Introduction

Je vais ici tester les méthodes proposées par le notebook de référence.
Le but étant de m'acclimater aux outils proposés.

#### Importation des librairies utilisées plus bas

In [ ]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings
import warnings

warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

#### Chargement des deux datasets

In [ ]:
(app_train := get_dataset('application_train'))

In [ ]:
(app_test := get_dataset('application_test'))

#### Traitement des valeurs manquantes (déséquilibre TARGET déjà vu plus tôt)

In [ ]:
# Function to calculate missing values by column# Funct
def missing_values_table(df):
    # Total missing values
    mis_val = df.isnull().sum()

    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)

    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
        columns={0: 'Missing Values', 1: '% of Total Values'})

    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:, 1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)

    # Print some summary information
    print("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
                                                              "There are " + str(mis_val_table_ren_columns.shape[0]) +
          " columns that have missing values.")

    # Return the dataframe with missing information
    return mis_val_table_ren_columns


# Missing values statistics
(missing_values := missing_values_table(app_train))

In [ ]:
for fold in ('train', 'test'):
    missing_test_values = missing_values_table(get_dataset('application_' + fold + '.csv'))

    # TODO: Set the plot style for dark mode when exporting to png
    plt.figure(figsize=(16, 12))  # There are a lot of columns
    sns.barplot(x=missing_test_values['% of Total Values'], y=missing_test_values.index)
    plt.title(f'Percentage of Missing Values by Feature ({fold.title()}ing fold)', fontsize=16)
    plt.xlabel('% of Total Values', fontsize=12)
    plt.ylabel('Features', fontsize=12)

    # Add percentage text on the bars
    for index, value in enumerate(missing_test_values['% of Total Values']):
        plt.text(value, index, f' {value}%', va='center')

    plt.xlim(0, 110)  # Set x-limit to give space for text
    plt.tight_layout()
    plt.savefig('test.png')
    plt.show()

In [ ]:
import missingno as msno

for fold in ('train', 'test'):
    msno.matrix(get_dataset('application_' + fold + '.csv'), fontsize=12)
    plt.title(f'Missing Values Count ({fold.title()}ing fold)', fontsize=16)
    plt.show()

In [ ]:
import missingno as msno

for fold in ('train', 'test'):
    df = get_dataset('application_' + fold + '.csv')
    msno.matrix(df[list(sorted(df.columns, key=(lambda col: int(df[col].notna().sum()))))], fontsize=12)
    plt.title(f'Missing Values Count ({fold.title()}ing fold)', fontsize=16)
    plt.show()

On peut voir ce qui a été montré précédemment, la moitié des colonnes ne possèdent aucunes valeurs manquantes, et le reste va grossièrement de 40 à 60 pourcent de valeurs manquantes

In [ ]:
# TODO: The notebook suggests XGBoost to get away with the use of a dataset with missing values, I should probably look for all the models and split them into two groups, one that I will use first after filling the voids, and the second one that will get the unprocessed dataset, to see if the imputation adds or reduces accuracy

In [ ]:
train_test_diff_col, = set(app_train) - set(app_test)
display(Markdown(f'La colonne **{train_test_diff_col!r}** est la seule à manquer du dataset de test'))

In [ ]:
assert not app_train.TARGET.isna().sum()
display(Markdown('Il ne manque aucune target'))

#### Types de données

Les colonnes étant communes entre train et test, je vais me contenter de voir les types des valeurs de training

TODO: Combiner les deux durant l'analyse, ou garder une séparation stricte?

In [ ]:
# Number of each type of column
app_train.dtypes.value_counts()

In [ ]:
app_train[app_train.columns[app_train.dtypes == 'float64']]

In [ ]:
app_train[app_train.columns[app_train.dtypes == 'int64']]

Le type objet peut être associée avec une colonne numérique possédant quelques valeurs invalides, il est donc important de m'assurer que ce n'est pas le cas ici

In [ ]:
app_train[app_train.columns[app_train.dtypes == 'object']]

In [ ]:
app_train[app_train.columns[app_train.dtypes == 'object']].value_counts()

In [ ]:
pd.DataFrame(
    data=[[col_name, len(app_train[col_name].unique())] for col_name in
          app_train.columns[app_train.dtypes == 'object']],
    columns=['column', 'unique_count'])

In [ ]:
tmp = pd.DataFrame(
    data=[[col_name, len(app_train[col_name].unique())] for col_name in
          app_train.columns[app_train.dtypes == 'object']],
    columns=['column', 'unique_count'])
# TODO: Set the plot style for dark mode when exporting to png
plt.figure(figsize=(16, 12))  # There are a lot of columns
sns.barplot(x='unique_count', y='column', data=tmp)
plt.title(f'Percentage of Missing Values by Feature ({fold.title()}ing fold)', fontsize=16)
plt.xlabel('% of Total Values', fontsize=12)
plt.ylabel('Features', fontsize=12)


On peut voir relativement peu de types de valeurs textuelle, ce qui est bon si nous décitons d'utiliser des méthodes comme OneHot<br>

TODO: Voir s'il est possible de transformer les flags et booleans en int<br>
TODO: Voir s'il est possible de transformer les code gender -1 et 1 pour homme/femme et 0 pour XNA<br>
TODO: Voir d'autres algorithmes à utiliser pour transformer texte en nombre<br>
TODO: Voir la liste des algo supportant les données textuelles (je crois que RandomForest supporte)<br>
TODO: Should we do a PCA over the OneHot encoded organization type / occupation type?

In [ ]:
assert not len(app_train.columns[
                   (app_train.dtypes != 'int64') &
                   (app_train.dtypes != 'float64') &
                   (app_train.dtypes != 'object')]), 'Plus de types de colonnes sont présentes'

In [ ]:
num_train = app_train.copy()
num_test = app_test.copy()
del app_train, app_test

# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in num_train:
    if num_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(num_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(num_train[col])
            # Transform both training and testing data
            num_train[col] = le.transform(num_train[col])
            num_test[col] = le.transform(num_test[col])

            # Keep track of how many columns were label encoded
            le_count += 1

# one-hot encoding of categorical variables
num_train = pd.get_dummies(num_train)
num_test = pd.get_dummies(num_test)

print('Training Features shape: ', num_train.shape)
print('Testing Features shape: ', num_test.shape)

print('%d columns were label encoded.' % le_count)

train_labels = num_train['TARGET']

# Align the training and testing data, keep only columns present in both dataframes
num_train, num_test = num_train.align(num_test, join='inner', axis=1)

# Add the target back in
num_train['TARGET'] = train_labels

print('Training Features shape: ', num_train.shape)
print('Testing Features shape: ', num_test.shape)

In [ ]:
((num_train['DAYS_BIRTH'] / -365).describe()).to_frame().join(
    (num_test['DAYS_BIRTH'] / -365).describe(),
    lsuffix='_train',
    rsuffix='_test'
)

TODO: Est-ce normal d'avoir la même chose?

In [ ]:
((num_train['DAYS_BIRTH'] / -365).describe()).to_frame().reset_index()

In [ ]:
((num_test['DAYS_BIRTH'] / -365).describe()).to_frame().reset_index()

In [ ]:
((num_train['DAYS_EMPLOYED']).describe()).to_frame().join(
    (num_test['DAYS_EMPLOYED']).describe(),
    lsuffix='_train',
    rsuffix='_test'
)

In [ ]:
num_train['DAYS_EMPLOYED'].plot.hist(title='Days Employment Histogram (Train)')
plt.xlabel('Days Employment')
plt.show()
num_test['DAYS_EMPLOYED'].plot.hist(title='Days Employment Histogram (Test)')
plt.xlabel('Days Employment')
plt.show()

In [ ]:
anom = num_train[num_train['DAYS_EMPLOYED'] == 365243]
non_anom = num_train[num_train['DAYS_EMPLOYED'] != 365243]
print('The non-anomalies default on %0.2f%% of loans' % (100 * non_anom['TARGET'].mean()))
print('The anomalies default on %0.2f%% of loans' % (100 * anom['TARGET'].mean()))
print('There are %d anomalous days of employment' % len(anom))

In [ ]:
# TODO: Set a new name
app_train, app_test = num_train.copy(), num_test.copy()
del num_train, num_test

# Create an anomalous flag column
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243

# Replace the anomalous values with nan
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace=True)

app_train['DAYS_EMPLOYED'].plot.hist(title='Days Employment Histogram')
plt.xlabel('Days Employment')

app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace=True)
print(
    'There are %d anomalies in the test data out of %d entries' % (app_test["DAYS_EMPLOYED_ANOM"].sum(), len(app_test)))

In [ ]:
# Find correlations with the target and sort
correlations = app_train.corr()['TARGET'].sort_values()

# Display correlations
print('Most Positive Correlations:\n')
correlations.tail(15)

In [ ]:
print('\nMost Negative Correlations:\n')
correlations.head(15)

In [ ]:
# Find the correlation of the positive days since birth and target
assert (app_train['DAYS_BIRTH'] > 0).sum() == 0, (
    (app_train['DAYS_BIRTH'] < 0).sum())
app_train['DAYS_BIRTH'].corr(app_train['TARGET'])

This is the most correlated to the target.
The days birth being negative, a positive corellation means that young people are less likely to repay on time

In [ ]:
# Set the style of plots
plt.style.use('fivethirtyeight')

# Plot the distribution of ages in years
plt.hist(app_train['DAYS_BIRTH'] / -365, edgecolor='k', bins=25)
plt.title('Age of Client');
plt.xlabel('Age (years)');
plt.ylabel('Count');

In [ ]:
plt.figure(figsize=(10, 8))

# KDE plot of loans that were repaid on time
sns.kdeplot(app_train.loc[app_train['TARGET'] == 0, 'DAYS_BIRTH'] / -365, label='target == 0')

# KDE plot of loans which were not repaid on time
sns.kdeplot(app_train.loc[app_train['TARGET'] == 1, 'DAYS_BIRTH'] / -365, label='target == 1')

# Labeling of plot
plt.xlabel('Age (years)');
plt.ylabel('Density');
plt.title('Distribution of Ages');

In [ ]:
# Age information into a separate dataframe
age_data = app_train[['TARGET', 'DAYS_BIRTH']]
age_data['YEARS_BIRTH'] = age_data['DAYS_BIRTH'] / -365

# Bin the age data
age_data['YEARS_BINNED'] = pd.cut(age_data['YEARS_BIRTH'], bins=np.linspace(20, 70, num=22))
display(age_data.head(5))

# Group by the bin and calculate averages
age_groups = age_data.groupby('YEARS_BINNED').mean()
display(age_groups)

plt.figure(figsize=(8, 8))

# Graph the age bins and the average of the target as a bar plot
plt.bar(age_groups.index.astype(str), 100 * age_groups['TARGET'])

# Plot labeling
plt.xticks(rotation=75);
plt.xlabel('Age Group (years)');
plt.ylabel('Failure to Repay (%)')
plt.title('Failure to Repay by Age Group')

In [ ]:
# Extract the EXT_SOURCE variables and show correlations, as those are the three least correlated
ext_data = app_train[['TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]
ext_data_corrs = ext_data.corr()
ext_data_corrs

In [ ]:
plt.figure(figsize=(8, 6))

# Heatmap of correlations
sns.heatmap(ext_data_corrs, cmap=plt.cm.RdYlBu_r, vmin=-0.25, annot=True, vmax=0.6)
plt.title('Correlation Heatmap');

The biggest correlation is between the first external source and the age

In [ ]:
plt.figure(figsize=(10, 12))

# iterate through the sources
for i, source in enumerate(['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']):
    # create a new subplot for each source
    plt.subplot(3, 1, i + 1)
    # plot repaid loans
    sns.kdeplot(app_train.loc[app_train['TARGET'] == 0, source], label='target == 0')
    # plot loans that were not repaid
    sns.kdeplot(app_train.loc[app_train['TARGET'] == 1, source], label='target == 1')

    # Label the plots
    plt.title('Distribution of %s by Target Value' % source)
    plt.xlabel('%s' % source);
    plt.ylabel('Density');

plt.tight_layout(h_pad=2.5)


All are weak according to the most common interpretation of the pearson coefficient

- .00-.19 “very weak”
- .20-.39 “weak”
- .40-.59 “moderate”
- .60-.79 “strong”
- .80-1.0 “very strong”

In [ ]:
# Copy the data for plotting
plot_data = ext_data.drop(columns=['DAYS_BIRTH']).copy()

# Add in the age of the client in years
plot_data['YEARS_BIRTH'] = age_data['YEARS_BIRTH']

# Drop na values and limit to first 100000 rows
plot_data = plot_data.dropna().loc[:100000, :]


# Function to calculate correlation coefficient between two columns
def corr_func(x, y, **kwargs):
    r = np.corrcoef(x, y)[0][1]
    ax = plt.gca()
    ax.annotate("r = {:.2f}".format(r),
                xy=(.2, .8), xycoords=ax.transAxes,
                size=20)


# Create the pairgrid object
grid = sns.PairGrid(data=plot_data, diag_sharey=False,
                    hue='TARGET',
                    vars=[x for x in list(plot_data.columns) if x != 'TARGET'])

# Upper is a scatter plot
grid.map_upper(plt.scatter, alpha=0.2)

# This forces the old behavior, scaling each curve independently
grid.map_diag(sns.kdeplot, common_norm=False)

# Bottom is density plot
grid.map_lower(sns.kdeplot, cmap=plt.cm.OrRd_r)

plt.suptitle('Ext Source and Age Features Pairs Plot', size=32, y=1.05)

In [ ]:
app_train.head().to_csv()

# Data Leakage

On a vu plus tôt une très faible corrélation entre les variables et la TARGET.<br>
Un risque possible serai d'avoir une variable d'entrée définies après l'emprunt<br>
Cependant les variables EXT_SOURCE_x sont opaques
Et les colonnes *_SOCIAL_CIRCLE peuvent ne pas avoir été prises assez tôt

In [ ]:
app_train